In [1]:
import random
import json
import time
import requests
import pandas as pd

In [21]:
pd.options.display.max_columns = None


def str_to_dict(x):
    return eval('{"' + x.replace(": ", '":"').replace('\n', '","') + '"}')


headers = str_to_dict("""accept: */*
accept-encoding: gzip, deflate, br
accept-language: zh-CN,zh;q=0.9
referer: https://space.bilibili.com/6574487/fans/follow
sec-fetch-dest: script
sec-fetch-mode: no-cors
sec-fetch-site: same-site
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36""")


# 获取单个UP主的关注对象
def follows_spider(up_id):
    data = []
    page = 1
    # B站默认限制查看前100个关注者
    while page <= 5:
        time.sleep(random.randint(6, 12) * 0.1)
#         print(up_id, page)
        # 关注者api
        api_url = "http://api.bilibili.com/x/relation/followings?vmid=%s&pn=%s&ps=50&order=desc&jsonp=jsonp&callback=__jp5" % (
            up_id, page)
        response = requests.get(api_url, headers=headers)
        
        try:
            # 关注者
            follows = json.loads(response.text[6:-1])["data"]["list"]
        except Exception as e:
            print(e)
            print(json.loads(response.text[6:-1]))
            break

        data.extend(follows)
        page += 1
    return data


# BFS
# 广度优先搜索UP主的关注对象
def follow_tree_spider(seeds, deep=1):
    # 详细数据
    final_data = pd.DataFrame()
    # 已获取
    from_cache = set()
    # 待获取
    queue = set(seeds)
    
    try:
        for i in range(deep):
            print("正在获取第%级关注."%i)
            sub_data = pd.DataFrame()
            for seed in queue:
                sub_data = pd.DataFrame(follows_spider(str(int(float(seed)))))
                sub_data["from"] = seed
                final_data = final_data.append(sub_data)
            # 单层搜索结束进行缓存更新
            from_cache.update(queue)
            # 当前列表去除缓存数据
            # print(sub_data.columns)
            queue = set(final_data["mid"]) - from_cache
    except Exception as e:
        print(e)
    return final_data

In [22]:
headers

{'accept': '*/*',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'zh-CN,zh;q=0.9',
 'referer': 'https://space.bilibili.com/6574487/fans/follow',
 'sec-fetch-dest': 'script',
 'sec-fetch-mode': 'no-cors',
 'sec-fetch-site': 'same-site',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

In [23]:
seeds = ['6574487']
data = follow_tree_spider(seeds, 3)

351502018 1
351502018 2
351502018 3
351502018 4
351502018 5
9458053 1
9458053 2
9458053 3
9458053 4
9458053 5
1202762767 1
1202762767 2
1202762767 3
1202762767 4
1202762767 5
354188817 1
354188817 2
354188817 3
354188817 4
354188817 5
546195 1
546195 2
546195 3
546195 4
546195 5
6574487 1
6574487 2
6574487 3
6574487 4
6574487 5
186443421 1
186443421 2
186443421 3
186443421 4
186443421 5
113362335 1
113362335 2
113362335 3
113362335 4
113362335 5
250681504 1
250681504 2
250681504 3
250681504 4
250681504 5
7936287 1
7936287 2
7936287 3
7936287 4
7936287 5
254463269 1
254463269 2
254463269 3
254463269 4
254463269 5
194046502 1
194046502 2
194046502 3
194046502 4
194046502 5
4835752 1
4835752 2
4835752 3
4835752 4
4835752 5
491936052 1
491936052 2
491936052 3
491936052 4
491936052 5
10897973 1
10897973 2
10897973 3
10897973 4
10897973 5
91454396 1
91454396 2
91454396 3
91454396 4
91454396 5
147166910 1
147166910 2
147166910 3
147166910 4
147166910 5
483514946 1
483514946 2
483514946 3
4835

In [25]:
len(data)

8966

In [41]:
data["mid"] = data["mid"].astype(int)
data["from"] = data["from"].astype(int)

In [45]:
len(data.mid.unique())

5551

In [46]:
len(data["from"].unique())

89

In [53]:
seeds2 = list(set(data["mid"]) - set(data["from"]))[:10]

In [54]:
data2 = follow_tree_spider(seeds2, 2)

81922 1
81922 2
81922 3
81922 4
81922 5
44974083 1
44974083 2
44974083 3
44974083 4
44974083 5
9 1
9 2
9 3
9 4
9 5
12 1
12 2
12 3
12 4
12 5
31113228 1
31113228 2
31113228 3
31113228 4
31113228 5
77021198 1
77021198 2
77021198 3
77021198 4
77021198 5
6701071 1
6701071 2
6701071 3
6701071 4
6701071 5
16 1
16 2
16 3
16 4
16 5
37781521 1
37781521 2
37781521 3
37781521 4
37781521 5
5242903 1
5242903 2
5242903 3
5242903 4
5242903 5
1 1
1 2
1 3
1 4
1 5
2 1
2 2
2 3
2 4
2 5
369750017 1
369750017 2
369750017 3
369750017 4
369750017 5
4 1
4 2
4 3
4 4
4 5
93855752 1
93855752 2
93855752 3
93855752 4
93855752 5
382820360 1
382820360 2
382820360 3
382820360 4
382820360 5
18 1
18 2
18 3
18 4
18 5
2371603 1
2371603 2
2371603 3
2371603 4
2371603 5
20 1
20 2
20 3
20 4
20 5
745493 1
745493 2
745493 3
745493 4
745493 5
19 1
19 2
19 3
19 4
19 5
4034583 1
4034583 2
4034583 3
4034583 4
4034583 5
495419424 1
495419424 2
495419424 3
495419424 4
495419424 5
606243 1
606243 2
606243 3
606243 4
606243 5
12824612 1

In [27]:
data.to_excel("../data/follows_deep3_self.xlsx",index=True)

In [64]:
data2.to_csv("../data/follows_deep3_self2.csv",encoding="utf_8_sig")

In [63]:
len(data2)

62277